# Working of the blockchain

---

### 1. Blockchain as a Data Structure

A blockchain is essentially a **linked list of blocks**. Each block contains:
- A list of transactions
- Its own cryptographic hash
- The hash of the previous block

---

### 2. Hashing and Cryptographic Security

Each block's hash is calculated using the SHA-256 algorithm.

`block.hash = SHA256(block's_actual_contents)`

---

### 3. Chaining with Hashes

Each block stores the hash of the previous block, by creating the following recursive link:
`Block[i].previous_hash = Block[i-1].hash`

This forms a chain. If any block is tampered, its hash changes and this breaks the chain from that point onward.

---
### 4. Tampering and Chain Validation

If someone modifies the data in `Block[i]`, then:
- The hash of `Block[i]` changes.
- The `previous_hash` of `Block[i+1]` no longer matches.
- As a result, all subsequent blocks become invalid.

#### Chain Validation Checks:
1. Each block's `hash` must equal `calculate_hash()` (no tampering).
2. Each block's `previous_hash` must match the actual `hash` of the block before it.

---

### 5. Proof-of-Work (Mining)

Proof-of-Work is a computational puzzle solved during block creation.

#### Objective:
Find a `nonce` such that the hash of the block starts with a specific number of leading zeros (determined by `difficulty`):


This process requires trial and error and computation.

---

In [1]:
import hashlib
import time
from typing import List


class Block:
    def __init__(self, index: int, transactions: List[str], previous_hash: str, timestamp: float = None, nonce: int = 0):
        self.index = index
        self.timestamp = timestamp or time.time()
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = nonce
        self.hash = self.calculate_hash()

    def calculate_hash(self) -> str:
        block_string = f"{self.index}{self.timestamp}{self.transactions}{self.previous_hash}{self.nonce}"
        return hashlib.sha256(block_string.encode()).hexdigest()

    def __repr__(self):
        return (
            f"Block #{self.index}\n"
            f"Timestamp: {time.ctime(self.timestamp)}\n"
            f"Transactions: {self.transactions}\n"
            f"Previous Hash: {self.previous_hash}\n"
            f"Nonce: {self.nonce}\n"
            f"Hash: {self.hash}\n"
        )


class Blockchain:
    def __init__(self, difficulty: int = 4):
        self.chain: List[Block] = [self.create_genesis_block()]
        self.pending_transactions: List[str] = []
        self.difficulty = difficulty

    def create_genesis_block(self) -> Block:
        return Block(index=0, transactions=["Genesis Block"], previous_hash="0")

    def get_latest_block(self) -> Block:
        return self.chain[-1]

    def add_transaction(self, transaction: str):
        self.pending_transactions.append(transaction)

    def mine_pending_transactions(self):
        if not self.pending_transactions:
            print("No transactions to mine.")
            return

        new_block = Block(
            index=len(self.chain),
            transactions=self.pending_transactions.copy(),
            previous_hash=self.get_latest_block().hash)

        self.proof_of_work(new_block)
        self.chain.append(new_block)
        self.pending_transactions = []

    def proof_of_work(self, block: Block):
        print(f"Mining Block #{block.index}...")
        while not block.hash.startswith("0" * self.difficulty):
            block.nonce += 1
            block.hash = block.calculate_hash()
        print(f"Block #{block.index} mined: {block.hash}\n")

    def is_chain_valid(self) -> bool:
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i - 1]

            if current.hash != current.calculate_hash():
                print(f"Invalid hash at Block #{current.index}")
                return False

            if current.previous_hash != previous.hash:
                print(f"Invalid previous hash link at Block #{current.index}")
                return False
        return True

    def print_chain(self):
        for block in self.chain:
            print(block)

    def tamper_with_block(self, index: int, new_data: List[str]):
        if 0 < index < len(self.chain):
            print(f"Tampering with Block #{index}...")
            self.chain[index].transactions = new_data
            self.chain[index].hash = self.chain[index].calculate_hash()
        else:
            print("Invalid block index for tampering.")


In [4]:
# Implementation

blockchain = Blockchain(difficulty=3)

blockchain.add_transaction("Aditya pays Deep 5 BTC")
blockchain.add_transaction("Deep pays Ronak 2 BTC")
blockchain.mine_pending_transactions()

blockchain.add_transaction("Ronak pays Dave 3 BTC")
blockchain.add_transaction("Dave pays Aditya 1 BTC")
blockchain.mine_pending_transactions()

print("Blockchain")
blockchain.print_chain()

print("Validating Blockchain")
print("Is chain valid?", blockchain.is_chain_valid())

print("Tampering with Blockchain")
blockchain.tamper_with_block(1, ["Tampered Transaction!"])
blockchain.print_chain()

print("Re-validating Blockchain After Tampering")
print("Is chain valid?", blockchain.is_chain_valid())


Mining Block #1...
Block #1 mined: 0000418ef2756632e55032fa58f0d4ca465d0478e852ef0b932f29d9991936eb

Mining Block #2...
Block #2 mined: 00051531178ccd4f52f3be6cd2d6ac49560c1dbcfcbfd86632f97904adfcf875

Blockchain
Block #0
Timestamp: Wed Mar 26 11:18:29 2025
Transactions: ['Genesis Block']
Previous Hash: 0
Nonce: 0
Hash: c0c49627602f42b9476340468847286d451deca4b5b0cefd5c3bf0578e62eda0

Block #1
Timestamp: Wed Mar 26 11:18:29 2025
Transactions: ['Aditya pays Deep 5 BTC', 'Deep pays Ronak 2 BTC']
Previous Hash: c0c49627602f42b9476340468847286d451deca4b5b0cefd5c3bf0578e62eda0
Nonce: 765
Hash: 0000418ef2756632e55032fa58f0d4ca465d0478e852ef0b932f29d9991936eb

Block #2
Timestamp: Wed Mar 26 11:18:29 2025
Transactions: ['Ronak pays Dave 3 BTC', 'Dave pays Aditya 1 BTC']
Previous Hash: 0000418ef2756632e55032fa58f0d4ca465d0478e852ef0b932f29d9991936eb
Nonce: 5389
Hash: 00051531178ccd4f52f3be6cd2d6ac49560c1dbcfcbfd86632f97904adfcf875

Validating Blockchain
Is chain valid? True
Tampering with Block